In [7]:
import pyro
import numpy as np
import torch 

ModuleNotFoundError: No module named 'pyro.dist'

In [2]:
# Assuning that our data is conditionallly independent 
def model(data):
    # sample f from the beta prior
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    # loop over the observed data using pyro.sample with the obs keyword argument
    # for i in range(len(data)):
    
    # Because our data is conditionally independent we can replace the for loop for pyro's optimised for loop 
    # Basically pyro.plate is implemented using a context manager.
    # At every execution of the body of the for loop we enter a new (conditional) 
    # independence context which is then exited at the end of the for loop body.
    for i in pyro.plate("data_loop", len(data)):

        # observe datapoint i using the bernoulli likelihood
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])

In [3]:
data = torch.zeros(10)
data[0:6] = torch.ones(6)  # 6 heads and 4 tails

In [4]:
# Vectorised plate notation fr conditional independence 
with pyro.plate('observe_data'):
    pyro.sample('obs', dist.Bernoulli(f), obs=data)

NameError: name 'dist' is not defined

In [ ]:
# Deefine manually the subsample size 
for i in pyro.plate("data_loop", len(data), subsample_size=5):
    pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])
    
with plate('observe_data', size=10, subsample_size=5) as ind:
    pyro.sample('obs', dist.Bernoulli(f),
                obs=data.index_select(0, ind))

In [8]:
# local and global variables 
def model(data):
    beta = pyro.sample("beta", ...) # sample the global RV
    for i in pyro.plate("locals", len(data)):
        z_i = pyro.sample("z_{}".format(i), ...)
        # compute the parameter used to define the observation
        # likelihood using the local random variable
        theta_i = compute_something(z_i)
        pyro.sample("obs_{}".format(i), dist.MyDist(theta_i), obs=data[i])

In [9]:
def guide(data):
    beta = pyro.sample("beta", ...) # sample the global RV
    for i in pyro.plate("locals", len(data), subsample_size=5):
        # sample the local RVs
        pyro.sample("z_{}".format(i), ..., lambda_i)